# Algoritmos de Dijkstra y Prim

Ejecutar las celdas en orden, la última de todas contiene toda la interfaz para ejecutar cada uno de los ejercicios.

In [1]:
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import math
import random as rd

nom = {0:'MANZANARES', 1:'LA SOLANA', 2:'VILLARTA DE SAN JUAN', 3:'DAMIEL', 4:'ALMAGRO', 5:'ARGAMASILLA DE ALBA', 6:'MEMBRILLA', 7:'MORAL DE CALATRAVA', 8:'VALDEPEÑAS', 9:'VILLANUEVA DE LOS INFANTES', 10:'SAN CARLOS DEL VALLE', 11:'RUIDERA', 12:'TOMELLOSO', 13:'VILLARROBLEDO', 14:'VILLARUBIA DE LOS OJOS'}
ADY = pd.read_excel("GrafoQuixote.xlsx")

# Algoritmo de Dijkstra

In [2]:
def Dijkstra(ADY, origen, destino):
    
    numnodos = len(ADY)
    colaPrio = [] #contiene distancias
    recorrido = [] #nodos por los que voy pasando
    Dist = []
    nodosvisitados = []
    
    D = nx.Graph()

    #Variables auxiliares para origen y destino
    j = 0
    t = 0
    inf = math.inf
    
    for i in range(0, numnodos, +1):
        Dist.append(inf) #Dist lleno de infinitos
        colaPrio.insert(i,Dist[i])
    
    while(nom[j] != origen):
        j = j + 1
        
    Dist[j] = 0 #No hay ningún camino al origen
    colaPrio[j] = 0
    
    #print(Dist)
    #print(colaPrio)

    while (len(colaPrio) != 0):
        s = min(colaPrio)
        
        if s not in Dist: #Todos los nodos ya tienen una distancia asignada
            break
        else:

            v = Dist.index(s)

            colaPrio.remove(s)

            #print("s",s)
            #print("v",v)

            if v in nodosvisitados:
                continue

            else:

                vecinos = []
                for u in range(0, 15, +1):
                    if ADY[v][u] > -1:
                        vecinos.append(u)

                #print(vecinos)
                #print(Dist)

                for k in range(len(vecinos)):
                    w = vecinos[k]
                    if (Dist[v] + ADY[v][w] < Dist[w]):
                        Dist[w] = Dist[v] + ADY[v][w]
                        colaPrio.insert(w, Dist[w])

                nodosvisitados.append(v)

                #print(colaPrio)
    
    while(nom[t] != destino):
        t = t + 1
    
    coste = Dist[t] #Coste para ir de origen a destino
    
    return coste

# Algoritmo de Prim

In [3]:
def Prim(ADY, origen):
    
    numnodos = len(ADY)
    aristasvisitadas = []
    nodosvisitados = []
    
    H = nx.DiGraph()
    
    n = 0
    while(nom[n] != origen):
        n = n + 1
        
    nodo = n
    
    H.add_node(nodo)
    
    while len(nodosvisitados) != numnodos: #Hasta que no haya visitado los 15 nodos no para
        
        aristas = [] #Lista que contendrá todas las aristas vecinas al nodo
        pesos = [] #Lista con los pesos de las aristas vecinas al nodo de la que sacaremos el min
        arista = [] #Lista auxiliar para sacar la arista de menor peso que nos indicará a que nodo iremos
        
        if nodo not in nodosvisitados:  #Metemos el nodo en la lista de visitados si no ha sido visitado
            nodosvisitados.append(nodo)
            
            #print("Nodosvisitados", nodosvisitados)
        
        else:
            
            for i in range(0, numnodos, +1):

                if (ADY[nodo][i] > -1):

                    if [nodo,i] in aristasvisitadas:  #Comprobamos que no hayamos pasado por esa arista
                        continue

                    elif [i,nodo] in aristasvisitadas: #Grafo bidireccional, ojo
                        continue

                    else:
                        aristas.append([nodo,i])
                        pesos.append(ADY[nodo][i])
                        
            if len(aristas) == 0:  #Si llega a un camino sin salida o queda algún nodo marginado, visita nodos al azar hasta encontrar a su padre.
                nodo = rd.choice(nodosvisitados)
                
            else:
                #print(aristas)

                minimo = min(pesos)         #Sacamos la arista de valor min
                save = pesos.index(minimo)  #Guardamos la posición del minimo en pesos, para saber a que arista corresponde

                arista.append(aristas[save])
                nodo2 = arista[0][1] #Vamos al nodo "destino" -> [origen,destino]

                #print("Nodo2", nodo2)

                if nodo2 in nodosvisitados: #Comprobamos que no hayamos pasado ya por ese nodo
                    aristasvisitadas.append([nodo,nodo2])
                    nodo = n               #En caso de estar ya visitado pasamos a otro nodo

                else:
                    H.add_edges_from(([(nodo,nodo2)]), weigh = ADY[nodo][nodo2])  #Pintamos solo si el nodo siguiente no ha sido visitado
                    aristasvisitadas.append([nodo,nodo2])
                    nodo = nodo2

    print("\nA continuación se muestra la ruta para visitar todos los pueblos partiendo de %s: \n" % (origen))
    nx.draw.circular(H, node_color = 'w', with_labels = True)
    plt.show()

# Ruta de emergencias

In [4]:
def Emergencia(ADY, origen, emergencias, destino):
    
    if len(emergencias) == 0:
        print("No se ha detectado ninguna ruta de emergencia, calaculando la ruta desde %s hasta %s..." % (origen, destino))
        
        normal = Dijkstra(ADY, origen, destino)
        
        print("Distancia recorrida: %i km\n" % (normal))
            
    else:
        
        numemergencias = len(emergencias)
        
        if numemergencias == 3:
        
            emergencia1 = emergencias[0]
            print("Partimos de %s, pero surge una emergencia en %s. Recalculado ruta..." % (origen, emergencia1))

            coste1 = Dijkstra(ADY, origen, emergencia1)

            print("Estamos en %s habiendo recorrido %s km\n" % (emergencia1, coste1))

            emergencia2 = emergencias[1]

            print("Pero Dulcinea requiere de nuestra ayuda en %s, recalculando ruta..." % (emergencia2))

            coste2 = Dijkstra(ADY, emergencia1, emergencia2)

            coste12 = coste1 + coste2

            print("Nos encontramos ahora en %s con un coste acumulado de %i km\n" % (emergencia2, coste12))
            
            emergencia3 = emergencias[2]
            
            print("Surge una tercera emergencia en %s, calculando ruta..." % (emergencia3))
            
            coste3 = Dijkstra(ADY, emergencia2, emergencia3)
            
            coste23 = coste12 + coste3
            
            print("Nos encontramos ahora en %s habiendo completado en total %i km\n" % (emergencia3, coste23))
            
            costedestino = Dijkstra(ADY, emergencia3, destino)
            costefinal = coste23 + costedestino
            
            print("Finalizamos nuestras aventuras en %s con un recorrido total de %i km\n" % (destino, costefinal))
            
        elif numemergencias == 2:
        
            emergencia1 = emergencias[0]
            print("Partimos de %s, pero surge una emergencia en %s. Recalculado ruta..." % (origen, emergencia1))

            coste1 = Dijkstra(ADY, origen, emergencia1)

            print("Estamos en %s habiendo recorrido %s km\n" % (emergencia1, coste1))

            emergencia2 = emergencias[1]

            print("Pero Dulcinea requiere de nuestra ayuda en %s, recalculando ruta..." % (emergencia2))

            coste2 = Dijkstra(ADY, emergencia1, emergencia2)

            coste12 = coste1 + coste2

            print("Nos encontramos ahora en %s con un coste acumulado de %i km\n" % (emergencia2, coste12))
            
            costedestino = Dijkstra(ADY, emergencia2, destino)
            costefinal = coste12 + costedestino
            
            print("Finalizamos nuestras aventuras en %s con un recorrido total de %i km\n" % (destino, costefinal))
            
        elif numemergencias == 1:
        
            emergencia1 = emergencias[0]
            print("Partimos de %s, pero surge una emergencia en %s. Recalculado ruta..." % (origen, emergencia1))

            coste1 = Dijkstra(ADY, origen, emergencia1)

            print("Estamos en %s habiendo recorrido %s km\n" % (emergencia1, coste1))
            
            costedestino = Dijkstra(ADY, emergencia1, destino)
            costefinal = coste1 + costedestino
            
            print("Finalizamos nuestras aventuras en %s con un recorrido total de %i km\n" % (destino, costefinal))

# Interfaz de usuario

In [5]:
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import math

def menu():
    print("1- Ver el mapa")
    print("2- Ruta más óptima entre dos pueblos (Dijkstra)")
    print("3- Ruta alternativa que conecte todos los pueblos (Prim)")
    print("4- Ruta de emergencias")
    print("5- Salir")
    opc = int(input("Introduzca una opción: "))
    return opc

opc=menu()
while opc < 5:
    if opc == 1:
        
        print("\nMapa de los pueblos manchegos: \n")
        #nom = pd.read_excel('GrafoQuixote.xlsx', 'Hoja2')
        G = nx.DiGraph()
        nom = {0:'MANZANARES', 1:'LA SOLANA', 2:'VILLARTA DE SAN JUAN', 3:'DAMIEL', 4:'ALMAGRO', 5:'ARGAMASILLA DE ALBA', 6:'MEMBRILLA', 7:'MORAL DE CALATRAVA', 8:'VALDEPEÑAS', 9:'VILLANUEVA DE LOS INFANTES', 10:'SAN CARLOS DEL VALLE', 11:'RUIDERA', 12:'TOMELLOSO', 13:'VILLARROBLEDO', 14:'VILLARUBIA DE LOS OJOS'}

        ADY = pd.read_excel("GrafoQuixote.xlsx")
        for i in range (0, 15, +1):
            for j in range(0, 15, +1):
                if (ADY[i][j] > 0):
                    G.add_edges_from([(nom[i], nom[j])], weigh = ADY[i][j])
            

        nx.draw(G, node_color = 'w', with_labels = True)
        plt.show()
        
    elif opc==2:
        
        origen = input("Introduzca el pueblo de origen: ")
        while origen not in nom.values():
            print("Ese pueblo no existe, introduce otro.")
            origen = input("Introduzca el pueblo de origen: ")
        
        print("\nEmpezamos en: " + origen)

        destino = input("Introduzca el pueblo de destino: ")
        while destino not in nom.values():
            print("Ese pueblo no existe, introduce otro.")
            destino = input("Introduzca el pueblo de destino: ")
        
        print("\nFinalizamos en: " + destino)
        
        coste = Dijkstra(ADY, origen, destino)
        
        print("\n Sancho: Según mis cálculos, la distancia desde %s hasta %s es de %s Km \n" % (origen, destino, coste))
            
    elif(opc==3):
        
        origen = input("Introduzca el pubeblo de partida: ")
        while origen not in nom.values():
            print("Ese pueblo no existe, introduce otro.")
            origen = input("Introduzca el pueblo de partida: ")
        
        print("\nEmpezamos en: " + origen)
        
        Prim(ADY, origen)
        
    elif(opc==4):
        
        k = 0
        emergencias = []
        
        origen = input("Introduzca el pubeblo de partida: ")
        while origen not in nom.values():
            print("Ese pueblo no existe, introduce otro.")
            origen = input("Introduzca el pueblo de partida: ")
            
        nemergencias = int(input("Introduzca el número de emergencias (Max 3): "))
        while nemergencias > 3:
            print("D. Quijote soporta un máximo de 3 emergencias.")
            nemergencias = int(input("Introduzca el número de emergencias (Max 3): "))
        
        e1 = input("Introduzca el primer pueblo de emergencia: ")
        while e1 not in nom.values():
            print("Ese pueblo no existe, introduce otro.")
            e1 = input("Introduzca el primer pueblo de emergencia: ")
        emergencias.append(e1)
        k = k + 1
        
        if k == nemergencias:
            destino = input("Introduzca el pubeblo de destino\n: ")
            while destino not in nom.values():
                print("Ese pueblo no existe, introduce otro.")
                destino = input("Introduzca el pueblo de destino\n: ")
            

        else:
            e2 = input("Introduzca el segundo pueblo de emergencia: ")
            while e2 not in nom.values():
                print("Ese pueblo no existe, introduce otro.")
                e2 = input("Introduzca el segundo pueblo de emergencia: ")
            emergencias.append(e2)
            k = k + 1
            
            if k == nemergencias:
                destino = input("Introduzca el pubeblo de destino: ")
                while destino not in nom.values():
                    print("Ese pueblo no existe, introduce otro.")
                    destino = input("Introduzca el pueblo de destino: ")
            else:
                e3 = input("Introduzca el tercer pueblo de emergencia: ")
                while e3 not in nom.values():
                    print("Ese pueblo no existe, introduce otro.")
                    e3 = input("Introduzca el tercer pueblo de emergencia: ")
                emergencias.append(e3)
                k = k + 1
        
        Emergencia(ADY, origen, emergencias, destino)
        
    elif (opc == 5):
        
        print("Saliendo del menu...")
        
    opc= menu()

1- Ver el mapa
2- Ruta más óptima entre dos pueblos (Dijkstra)
3- Ruta alternativa que conecte todos los pueblos (Prim)
4- Ruta de emergencias
5- Salir
Introduzca una opción: 4
Introduzca el pubeblo de partida: ALMAGRO
Introduzca el número de emergencias (Max 3): 3
Introduzca el primer pueblo de emergencia: VILLANUEVA DE LOS INFANTES
Introduzca el segundo pueblo de emergencia: VILLARRUBIA
Ese pueblo no existe, introduce otro.
Introduzca el segundo pueblo de emergencia: VILLARUBIA DE LOS OJOS
Introduzca el tercer pueblo de emergencia: MANZANARES


NameError: name 'destino' is not defined